## Content analysis stuff

In [ ]:
from nlp_functions import * 
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline
# Example usage:
updated_df = extract_named_entities_hf_split('test-data\Bellingcat-regions-india-subcontinent.json', 'content')
updated_df.to_csv('ind_channel.csv', index=False)

In [1]:
file_path = 'ind_channel.csv'
column_name = 'content'
output_file_path = 'output_file_ncs3.csv'

In [2]:
# Noun phrase exraction (individual table)
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
from textblob import TextBlob
# Function to extract noun phrases
def extract_noun_phrases(text):
    blob = TextBlob(text)
    return [np for np in blob.noun_phrases]

# Function to process CSV and extract noun phrases
def extract_noun_chunks_from_csv1(file_path, column_name):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the CSV file.")
    
    # Extract text from the specified column
    texts = df[column_name].dropna().tolist()

    # Initialize a list to store the noun chunks
    all_noun_chunks = []

    # Process each text entry
    for text in texts:
        noun_chunks = extract_noun_phrases(text)
        all_noun_chunks.append(noun_chunks)
    
    # Create a new DataFrame with the extracted noun chunks
    result_df = pd.DataFrame({
        column_name: texts,
        'noun_chunks': all_noun_chunks
    })

    return result_df

# Example usage:
result_df = extract_noun_chunks_from_csv1(file_path, column_name)
result_df.head(20)


C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,content,noun_chunks
0,Quick question: I'm looking for a relatively s...,"[quick, interview subjects, various fascist co..."
1,ahh that's a good one,[]
2,@gare?,[@ gare]
3,i mean search operators are kinda the best too...,[search operators]
4,if they got a facebook you can check their lik...,[]
5,https://www.youtube.com/watch?v=Er8JPvdW7Zc,[]
6,Summary:\nRioter was doing a HH,"[summary, rioter, hh]"
7,https://www.bellingcat.com/news/americas/2021/...,[bellingcat article]
8,Same one that tried to send the laptop to the ...,[]
9,"allegedly tried to send the laptop, yeah",[]


In [3]:
# Function to extract noun phrases
def extract_noun_phrases(text):
    blob = TextBlob(text)
    return [np for np in blob.noun_phrases]

# Function to process CSV and extract noun phrases
def extract_noun_chunks_from_csv(file_path, column_name, output_file_path):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Ensure the column exists
    if column_name not in df.columns:
        raise ValueError(f"Column '{column_name}' not found in the CSV file.")
    
    # Extract text from the specified column
    texts = df[column_name].dropna().tolist()

    # Initialize a list to store the noun chunks
    all_noun_chunks = []

    # Process each text entry
    for text in texts:
        noun_chunks = extract_noun_phrases(text)
        noun_chunks_str = ', '.join(noun_chunks)  # Convert list to comma-separated string
        all_noun_chunks.append(noun_chunks_str)
    
    # Append noun_chunks to the original DataFrame
    df['noun_chunks'] = pd.Series(all_noun_chunks)

    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_file_path, index=False)

# Example usage:
extract_noun_chunks_from_csv(file_path, column_name, output_file_path)
print(f"Updated CSV file saved to {output_file_path}")




Updated CSV file saved to output_file_ncs3.csv


In [5]:
# Count noun phrases 
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords

# Define custom stopwords
custom_stopwords = [
    'i', 'im', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself',
    'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself',
    'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that',
    'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
    'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as',
    'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through',
    'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',
    'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not',
    'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should',
    'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', 'couldn', 'didn', 'doesn', 'hadn', 'hasn',
    'haven', 'isn', 'ma', 'mightn', 'mustn', 'needn', 'shan', 'shouldn', 'wasn', 'weren', 'won', 'wouldn',
    'm', 'dms', 'nytimes', 'bbc', 'apnews', 'outlookindia','two','10','000','##'
]

# Load the stopwords from NLTK
nltk_stopwords = set(stopwords.words('english'))
# Combine NLTK stopwords with custom stopwords
stop_words = nltk_stopwords.union(set(custom_stopwords))

def is_valid_phrase(phrase):
    # Split the phrase into words
    words = phrase.split()
    # Check if the phrase contains any non-stopword words
    return any(word.lower() not in stop_words for word in words)

def count_noun_phrases(file_path, column_name):
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Extract noun phrases from the specified column
    noun_phrases = df[column_name].dropna().str.split(', ')

    # Flatten the list of noun phrases
    all_noun_phrases = [phrase for sublist in noun_phrases for phrase in sublist]

    # Filter out phrases that are empty or contain only stopwords
    valid_noun_phrases = [phrase for phrase in all_noun_phrases if is_valid_phrase(phrase)]

    # Count the frequency of each valid noun phrase
    noun_phrase_counts = Counter(valid_noun_phrases)

    # Convert to a DataFrame for better visualization
    noun_phrase_counts_df = pd.DataFrame(noun_phrase_counts.items(), columns=['Noun Phrase', 'Frequency'])
    noun_phrase_counts_df = noun_phrase_counts_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

    return noun_phrase_counts_df

# Example usage:
file_path = 'output_file_ncs3.csv'
column_name = 'noun_chunks'
noun_phrase_counts_df = count_noun_phrases(file_path, column_name)



#### Entity plots

In [17]:
import pandas as pd
import plotly.express as px
import ner_functions as nf
nf.plot_top_entities('data/output_file1.csv', ['Organization','Person'], top_n=25)
nf.plot_top_entities('data/output_file1.csv', ['Location'], top_n=25)
nf.plot_top_entities('data/output_file1.csv', ['Weapon'], top_n=25)

### Posts over time 

In [18]:
data = pd.read_csv('data/output_file1.csv')
nf.filter_and_plot(data,aggregation='monthly')
nf.filter_and_plot(data,regex='Weapon',aggregation='monthly',column='entity_types')

c:\Users\User\Desktop\Tooling\ner_functions.py:252: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

c:\Users\User\Desktop\Tooling\ner_functions.py:252: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



c:\Users\User\Desktop\Tooling\ner_functions.py:252: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



## Model testing space

In [ ]:
# Load the model and tokenizer
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "eventdata-utd/conflibert-named-entity-recognition"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for NER
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Sample text
text = "John Doe is a software engineer at OpenAI, and he lives in San Francisco."

# Perform NER
ner_results = ner_pipeline(text)

# Print the results
for entity in ner_results:
    print(entity)
    print(f"Entity: {entity['word']}, Label: {entity['entity']}")
